In [3]:
def compute_monitoring_metrics(system_type, y_true, y_pred):
    """
    Returns a list of 2 metrics depending on system_type:
    - classification -> [accuracy, f1]
    - regression     -> [mae, rmse]
    - ranking        -> [precision@k, recall@k]
    """

    system_type = system_type.lower().strip()

    # ---------------- CLASSIFICATION ----------------
    if system_type == "classification":
        correct = 0
        for yt, yp in zip(y_true, y_pred):
            if yt == yp:
                correct += 1
        accuracy = correct / len(y_true) if len(y_true) != 0 else 0.0

        tp = fp = fn = 0
        for yt, yp in zip(y_true, y_pred):
            if yt == 1 and yp == 1:
                tp += 1
            elif yt == 0 and yp == 1:
                fp += 1
            elif yt == 1 and yp == 0:
                fn += 1

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0.0

        return [accuracy, f1]

    # ---------------- REGRESSION ----------------
    elif system_type == "regression":
        n = len(y_true)
        if n == 0:
            return [0.0, 0.0]

        abs_sum = 0.0
        sq_sum = 0.0

        for yt, yp in zip(y_true, y_pred):
            err = yt - yp
            abs_sum += abs(err)
            sq_sum += err * err

        mae = abs_sum / n
        rmse = (sq_sum / n) ** 0.5

        return [mae, rmse]

    # ---------------- RANKING ----------------
    elif system_type == "ranking":
        n = len(y_true)
        if n == 0:
            return [0.0, 0.0]

        k = min(5, n)

        # pair up (pred_score, true_label)
        pairs = list(zip(y_pred, y_true))

        # sort by predicted score descending
        pairs.sort(key=lambda x: x[0], reverse=True)

        # take top-k
        top_k = pairs[:k]

        relevant_in_top_k = 0
        total_relevant = 0

        for _, label in pairs:
            if label == 1:
                total_relevant += 1

        for _, label in top_k:
            if label == 1:
                relevant_in_top_k += 1

        precision_at_k = relevant_in_top_k / k if k != 0 else 0.0
        recall_at_k = relevant_in_top_k / total_relevant if total_relevant != 0 else 0.0

        return [precision_at_k, recall_at_k]

    else:
        raise ValueError("system_type must be either 'classification', 'regression', or 'ranking'")
